In [1]:
import bs4 as bs
import requests
import datetime
import yfinance as yf

In [2]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [5]:
tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]

In [7]:
start_date = '2018-01-01'
end_date = '2023-10-01'

In [8]:
data = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  503 of 503 completed


3 Failed downloads:
['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1514782800, endDate = 1696132800")
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2018-01-01 -> 2023-10-01)')
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


In [9]:
df = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
df.set_index('Date', inplace=True)

In [10]:
df

,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2018-01-02,A,64.625565,67.599998,67.889999,67.339996,67.419998,1047800.0
2018-01-03,A,66.269897,69.320000,69.489998,67.599998,67.620003,1698900.0
2018-01-04,A,65.772774,68.800003,69.820000,68.779999,69.540001,2230700.0
2018-01-05,A,66.824364,69.900002,70.099998,68.730003,68.730003,1632500.0
2018-01-08,A,66.967758,70.050003,70.330002,69.550003,69.730003,1613400.0
...,...,...,...,...,...,...,...
2023-09-25,ZTS,176.894302,177.720001,177.919998,175.139999,175.800003,1551100.0
2023-09-26,ZTS,176.048248,176.869995,178.449997,176.270004,176.580002,1463200.0
2023-09-27,ZTS,173.868408,174.679993,178.190002,172.789993,177.729996,1450400.0


In [11]:
df.to_csv('./data/sp500_stocks.csv')